In [12]:
# !pip install beautifulsoup4

In [56]:
import re
import os
import json
import regex
import requests

from bs4 import BeautifulSoup

In [57]:
SCORES_DICT = dict()
PROBLEMS_ROOT = "../data/reshu_ege/"

# Task 2 parser (example):

In [31]:
source_url = "https://rus-ege.sdamgia.ru/test?theme=349&print=true"

In [32]:
page = requests.get(source_url)
text = page.text.replace("&shy;", "").replace("</i>", "").replace("<i>", "").replace("</b>", "").\
                        replace("<b>", "").replace("\t", " ")

soup = BeautifulSoup(text, 'html.parser')

In [33]:
# get all tasks as an html objects:
all_tasks = soup.findAll("div", {"class": "prob_maindiv"})

In [34]:
# get all answers and parse them:
answers = re.findall("Ответ:[\ \tА-ЯЁ0-9ёа-я|]+", text)
answers = list(map(lambda x: re.findall("[А-ЯЁ0-9ёа-я|]+", x.replace("Ответ:", ""))[0].split("|"), answers))

In [35]:
# get all serial numbers:
task_serial_number = re.findall("problem\?id=[0-9]*", text)

In [36]:
# parse tasks (CHANGE CODE HERE):

questions_problems = []
questions_texts = []
questions_serial_numbers = []

for id_, task_ in enumerate(all_tasks):
    serial_number_ = task_serial_number = re.findall("problem\?id=[0-9]*", str(task_))
    problem_ = list(list(list(task_.children)[1].children)[0])[0]
    if len(problem_) == 1:
        problem_ = list(list(list(list(task_.children)[1].children)[0])[1].children)[0]

    text_ = list(list(list(list(task_.children)[3].children)[0])[0].children)[0].text.replace("\xad", "")

    problem_ = problem_.replace("\xa0", " ")
    text_ = text_.replace("\xa0", " ")

    questions_problems.append(problem_)
    questions_texts.append(text_)
    questions_serial_numbers.append(serial_number_)
    
questions_answers = answers

In [37]:
# you need to check at least length of all intance lists:

In [38]:
print(len(questions_answers), len(questions_answers), len(questions_problems), len(questions_texts))

82 82 82 82


In [52]:
TASK_ID = "2"

In [53]:
SCORES_DICT[TASK_ID] = 1

In [54]:
# form tasks according to the structure:

final_tasks = []

for i in range(len(questions_answers)):
    final_task = dict()
    final_task["id"] = TASK_ID
    final_task["meta"] = {
        "language": "rus",
        "source": "site:" + source_url + "__________" + "id:" + questions_serial_numbers[i][0]
    }
    final_task["text"] = ("Прочитайте текст и выполните задание." + "\n" +
                          questions_texts[i] + "\n" + 
                          questions_problems[i] + "\n"
                          "Запишите это слово.")

    final_task["question"] = {
        "type": "text",
        "max_length": 30,
        "recommended_length": 20,
        "restriction": "word"
      }
    
    sol_field_name = "correct_variants" if len(questions_answers[i]) > 1 else "correct"
    
    if len(questions_answers[i]) > 1:
        answer = questions_answers[i]
    else:
        answer = questions_answers[i][0]
    final_task["solution"] = {
        sol_field_name: answer
    }
    
    final_task["score"] = SCORES_DICT[final_task["id"]]

    final_tasks.append(final_task)

In [55]:
# check one task:

final_tasks[-10]

{'id': '2',
 'meta': {'language': 'rus',
  'source': 'site:https://rus-ege.sdamgia.ru/test?theme=349&print=true__________id:problem?id=14922'},
 'text': 'Прочитайте текст и выполните задание.\n(1)Если над человеческим ухом пропоёт петух, человек на время оглохнет: мощность петушиного крика составляет 130 децибел — то же самое будет, если стоять в пятнадцати метрах от взлетающего реактивного самолёта. (2)Однако петухи не теряют слух от собственного пения, <…> в тот момент, когда петух открывает клюв для крика, наружный слуховой проход у него сужается примерно на четверть, а мягкие ткани наполовину прикрывают барабанную перепонку. (3)У кур слуховой проход устроен немного иначе, и «заткнуть уши» они не могут, но потерять слух от близкого соседства с петухами им не грозит: куры слушают петухов не вплотную, к тому же у птиц слуховые рецепторы довольно быстро регенерируются.\nСамостоятельно подберите подчинительный союз, который должен стоять на месте пропуска во втором (2) предложении текст

In [63]:
# create dirs for task:

tasks_directory = os.path.join(PROBLEMS_ROOT, "task_" + TASK_ID)

if not os.path.exists(tasks_directory):
    os.makedirs(tasks_directory)
else:
    print("dir already exists")

dir already exists


In [69]:
# save all the tasks to the dir:

for id_, task_ in enumerate(final_tasks):
    file_with_task = {
        "tasks": [
            task_
        ]
    }
    task_filename = re.findall("[0-9]+", questions_serial_numbers[id_][0])[0] + ".json"
    task_path = os.path.join(tasks_directory, task_filename)
    if not os.path.exists(task_path):
        json.dump(file_with_task, open(task_path, "w"), ensure_ascii=False, indent=4)
    else:
        raise BaseException("Task already exists, comment this  check if needed")

BaseException: Task already exists, comment above check if needed